In [1]:
library(MASS)
library(fitHeavyTail)
library(xts)
library(quantmod)
library(igraph)
library(readr)
library(spectralGraphTopology)
library(combinat)
library(ggplot2)
library(reshape2)
library(ktvgraph)

library(Matrix)
library(LaplacesDemon)
library(expm)

source("R/utils.R")
source("R/learn_kcomp_heavytail_TV_graph_online.R")

options(warn = -1)

Warning message:
"package 'MASS' was built under R version 4.5.1"
Warning message:
"package 'fitHeavyTail' was built under R version 4.5.1"
Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Warning message:
"package 'quantmod' was built under R version 4.5.1"
Loading required package: TTR

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Warning message:
"package 'igraph' was built under R version 4.5.1"

Attaching package: 'igraph'


The following objects are masked from 'package:stats':

    decompose, spectrum


The following object is masked from 'package:base':

    union



Attaching package: 'spectralGraphTopology'


The following object is masked from 'package:stats':

    D



Attaching package: 'combinat'


The following object is masked from 'package:utils':

    combn


Warning message:
"package 'ggplot2' was built under R version 

In [2]:
set.seed(42)


pdf(file = "Plots_TVGL_results.pdf")


# number of sectors
q <- 1
winLen <-  200

# number of stocks
r <- 100
#log_returns <- read.csv("log_rw_returns.csv")
log_returns <- read.csv("log_returns.csv")

Nday <- nrow(log_returns) 

Nwin <- Nday%/% winLen  


Nday <- nrow(log_returns) 

Nwin <- Nday%/% winLen  




#----------------------------
## Online TV graph learning (proposed)
forget_fac <- 0.1
data_frame <- scale(log_returns[1:winLen,1:r])
#data_frame <- rw_t[1:winLen,]

S_cov <- cor(data_frame)

w <- spectralGraphTopology:::w_init('naive', MASS::ginv(S_cov))

w0 <- w
A0 <- A(w)
A0 <- A0 / rowSums(A0)
w0 <- spectralGraphTopology:::Ainv(A0)
w0 = w0/sum(w0)

w_lagged <- w0

w_lagged <- 0

graphs_list <- vector("list", Nwin)

components_graph <- vector("list", Nwin)

accuracy_vec <- rep(0,Nwin)
accuracy_adj_vec <- rep(0,Nwin)
purity_vec <- rep(0,Nwin)
mod_g_vec <- rep(0,Nwin)
mod_gt_vec <- rep(0,Nwin)
balanced_vec <- rep(0,Nwin)
balanced_norm_vec <- rep(0,Nwin)
GINI_vec <- rep(0,Nwin)
rank_mat <- rep(0,Nwin)

#cat("init w: ",w,"\n")
#cat("init w_lagged: ", w_lagged,"\n")
#cat("init w0: ",w0,"\n")

Cluster = TRUE

# heavy_type = "gaussian"
# heavy_type = "student"

head(data_frame)

print(cat("w0", w0))

res_mvt <- fit_mvt(scale(data_frame), nu = "MLE-diag-resampled") #Estimate parameters of a multivariate Student's t
nu <- res_mvt$nu 
cat("nu: ",nu,"\n")

,A,AAL,ABBV,ABC,ABMD,ABT,ADM,AEE,AEP,AES,⋯,DE,DFS,DGX,DHR,DLR,DOV,DRE,DTE,DUK,DVA
1,-0.2706328,-0.38601423,-0.28673217,1.1255779,-0.21450536,-0.009463173,0.5640626,1.008088039,0.7115951,0.6613222,⋯,0.2963741,-0.1672035,-0.05802069,-0.01788446,-0.01006728,-1.1256204,0.9910941,0.8794217,0.6289958,0.55701829
2,0.2571223,0.71744928,-0.02113479,-1.0335841,0.06350252,-0.532614709,-1.4113041,0.516217296,0.2966966,-1.9450751,⋯,-0.7139745,-1.3118348,-1.45654066,-0.45880772,0.54902296,-1.2529979,-1.2630663,-0.5158213,0.1562837,-1.07528468
3,-2.9480683,-0.77586550,-0.21298577,-2.2636938,-1.86305946,-1.544144308,-1.3814046,-0.007945391,-1.1281159,-1.4168692,⋯,-0.5714631,-1.2474217,-2.03009217,-0.92293496,-0.28957515,-0.8573711,-2.4400605,-1.1308538,-0.9467544,-0.59540075
4,-0.7478337,-0.07485781,-1.72006894,-0.3407103,1.30967554,-1.346642996,-0.2592677,-0.783191987,-0.1941153,0.6960179,⋯,-1.0238814,-0.8375706,-0.53693824,-0.05970597,0.14214055,-0.5229958,-1.8479018,-0.4982725,-0.2749274,0.09618228
5,-1.1779783,0.72008389,-2.00563503,-2.8534563,0.12618782,0.099603967,-0.4219023,-0.049568420,0.7599951,-0.6576122,⋯,0.2671903,0.6589600,0.59110465,-0.09643785,1.29476474,-0.8968196,0.5591078,0.8407736,1.0064780,-0.48157392
6,0.4004271,0.91310488,1.04670031,0.1698866,1.58846565,1.140201274,0.1275989,-0.365566785,-1.0048859,-1.7512894,⋯,0.2496829,0.6391955,0.30362926,-0.04135113,-0.55073136,-0.5348756,-1.1654866,0.4508651,-0.8119181,0.29728362


w0 0 0.0001005669 2.776454e-05 0.0004718587 0.0006544204 0 0.0005177618 0 0 0.0008335429 0.000695249 0.0005919373 0 0 0.0001259094 0.0005415873 0 0.0003005556 0.0005433196 0 7.564618e-05 4.332303e-05 0.0004807098 0 0.0003736326 0 6.178802e-06 0 0 0.0002623138 0 0 0 0.0007286157 0 0.0002348467 0 0 0 0.0007988116 0 6.739668e-06 0.0005213703 0.0002485096 0.0005149452 0.0003807322 0 0.0002406246 0.0001307742 6.182428e-05 0.001111168 8.038457e-05 0 0.0006351327 0 0 0.0002839929 0 0.0005012324 0 0 0.0002760693 9.509171e-05 0.0006078316 0.0001613762 0 0 0.0001583642 0 0.0002376187 0 0 0 6.756405e-05 0.0002010285 0.0001247512 0 0 3.07985e-05 0.000112521 0 0 2.095754e-05 0.0002181135 0 0.000669718 0.0009246919 0.001102446 0 0.0004400652 0 0 0.0002209217 0 0 0.0007995439 0 0.0005944541 0.0002448681 0 0.0002297607 0 7.317072e-05 0.0004984465 0.0007760416 0 0 0 0 0 0.0004810696 0 0 6.180783e-05 0.0009711807 0 0 0 0 0.0009086845 0.0002826093 0.0002896585 0.0005246092 2.041578e-05 0.0008697729 0 1.7

In [3]:

graphs_list <- learn_kcomp_heavytail_TV_graph_online(scale(data_frame),
                                                    k = q, # k the number of components of the graph. (Number of Clusters)
                                                    heavy_type = "student", # the statistical distribution of the data "gaussian" or "student".
                                                    nu = nu, # the degrees of freedom of the Student-t distribution. Must be a real number greater than 2.
                                                    sigma_e = exp(10), # hyperparameter that controls graph weight sparsity and time-consistency
                                                    w_lagged = w_lagged,
                                                    rho = 2, # ADMM hyperparameter.
                                                    d = 1, # the nodes' degrees. Either a vector or a single value.
                                                    w0 = w0, # initial value of the VAR coefficient
                                                    update_eta = TRUE, # whether to update eta during the optimization.
                                                    maxiter = 10,
                                                    verbose = TRUE)
graphs_list$weights

T_n 200NULL
sigma_e 22026.47NULL
alpha 4.539993e-07NULL
beta 0.1NULL
iter:  1NULL
iter:  2NULL
iter:  3NULL
iter:  4NULL
iter:  5NULL
iter:  6NULL
iter:  7NULL
iter:  8NULL
iter:  9NULL
iter:  10NULL
[1] "maxiter" "10"     


[1] 0.00000000 0.00000000 0.00000000 0.00000000 0.02962792 0.00000000
   [7] 0.00000000 0.00000000 0.00000000 0.00000000 0.02684933 0.00000000
  [13] 0.00000000 0.02504467 0.00000000 0.02781593 0.00000000 0.00000000
  [19] 0.03034428 0.00000000 0.00000000 0.02578942 0.03459889 0.02756536
  [25] 0.00000000 0.00000000 0.00000000 0.02887957 0.00000000 0.02822895
  [31] 0.00000000 0.00000000 0.00000000 0.02727485 0.00000000 0.00000000
  [37] 0.00000000 0.02572150 0.03005052 0.03046649 0.02834949 0.00000000
  [43] 0.00000000 0.02814110 0.00000000 0.02525254 0.00000000 0.00000000
  [49] 0.00000000 0.00000000 0.02731726 0.00000000 0.00000000 0.02862854
  [55] 0.00000000 0.00000000 0.02956395 0.00000000 0.02537552 0.00000000
  [61] 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000
  [67] 0.00000000 0.00000000 0.02569727 0.00000000 0.00000000 0.00000000
  [73] 0.00000000 0.00000000 0.02517103 0.00000000 0.00000000 0.00000000
  [79] 0.00000000 0.00000000 0.00000000 0.00000000 0.02677138 0.00000000
  [85] 0.00000000 0.00000000 0.00000000 0.03046063 0.00000000 0.01625102
  [91] 0.02577296 0.02959385 0.00000000 0.00000000 0.00000000 0.00000000
  [97] 0.00000000 0.00000000 0.00000000 0.01736671 0.01840692 0.01819260
 [103] 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000
 [109] 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000
 [115] 0.04917143 0.00000000 0.00000000 0.01913074 0.02189405 0.01594561
 [121] 0.00000000 0.00000000 0.00000000 0.01794258 0.00000000 0.00000000
 [127] 0.01929574 0.01601311 0.00000000 0.00000000 0.00000000 0.00000000
 [133] 0.00000000 0.02199440 0.01607224 0.01734352 0.00000000 0.00000000
 [139] 0.00000000 0.01616376 0.01769371 0.00000000 0.01949096 0.00000000
 [145] 0.00000000 0.01771631 0.00000000 0.00000000 0.01629044 0.00000000
 [151] 0.00000000 0.01704353 0.00000000 0.01669711 0.00000000 0.00000000
 [157] 0.00000000 0.01814320 0.02543406 0.01922629 0.00000000 0.01825005
 [163] 0.01879046 0.00000000 0.00000000 0.00000000 0.01695929 0.00000000
 [169] 0.02065468 0.00000000 0.00000000 0.00000000 0.01703940 0.01713071
 [175] 0.01755667 0.01834867 0.01676456 0.01709685 0.00000000 0.00000000
 [181] 0.01754713 0.01622766 0.00000000 0.00000000 0.00000000 0.07919736
 [187] 0.01671247 0.02046343 0.01714145 0.00000000 0.02783090 0.00000000
 [193] 0.01735964 0.00000000 0.00000000 0.00000000 0.00000000 0.01982705
 [199] 0.01939463 0.01754168 0.00000000 0.01995571 0.00000000 0.00000000
 [205] 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000
 [211] 0.01756322 0.01812022 0.00000000 0.00000000 0.02481243 0.03645610
 [217] 0.00000000 0.04195312 0.00000000 0.00000000 0.01940449 0.00000000
 [223] 0.00000000 0.01639031 0.00000000 0.00000000 0.00000000 0.00000000
 [229] 0.00000000 0.00000000 0.01854480 0.00000000 0.00000000 0.01691864
 [235] 0.01656864 0.00000000 0.02501293 0.01804334 0.00000000 0.00000000
 [241] 0.00000000 0.00000000 0.02386465 0.01834736 0.00000000 0.00000000
 [247] 0.00000000 0.01946751 0.00000000 0.00000000 0.01759630 0.00000000
 [253] 0.00000000 0.00000000 0.01942017 0.01862964 0.00000000 0.00000000
 [259] 0.01673644 0.02051390 0.00000000 0.00000000 0.00000000 0.00000000
 [265] 0.00000000 0.00000000 0.00000000 0.01838941 0.00000000 0.00000000
 [271] 0.01746400 0.01897094 0.01602017 0.01592819 0.01677993 0.01985619
 [277] 0.01722796 0.00000000 0.01588136 0.00000000 0.00000000 0.00000000
 [283] 0.01600979 0.00000000 0.01784889 0.00000000 0.00000000 0.02679534
 [289] 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.01632862
 [295] 0.01597598 0.01612456 0.00000000 0.01923443 0.01862999 0.00000000
 [301] 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000
 [307] 0.01693715 0.00000000 0.00000000 0.00000000 0.02023975 0.01921211
 [313] 0.00000000 0.01743268 0.00000000 0.00000000 0.01881837 0.00000000
 [319] 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000
 [325] 0.00000000 0.00000000 0.01755308 0.00000000 0.00000000

[1] 0.2260872 0.2828982 0.2054545 0.2855600 0.2340729 0.3032830 0.2365570
 [8] 0.2482042 0.2348247 0.2430582

In [15]:
graphs_list$weights

[1] 0.2259829 0.2828659 0.2057327 0.2854184 0.2342075 0.3031121 0.2366975
 [8] 0.2480988 0.2348278 0.2430563

In [ ]:
w <- c(0.1170013, 0.1527746, 0.04855342, 0.1085774, 0.1243674, 0.1150374, 0.09536096, 0.07961204, 0.08441811, 0.07429729)
L_n <- L(w)
print(L_n)
U <- eigen(L_n, symmetric = TRUE)$vectors
L <- eigen(L_n, symmetric = TRUE)$values
print("")
print(L)
print("")
print(U)


In [ ]:
# number of stocks
r <- 100 
stock_prices_orig <- readRDS("examples/stocks/sp500-data-2016-2020.rds")
stock_prices <- stock_prices_orig[1:1001,1:r]
log_returns <- diff(log(stock_prices), na.pad = FALSE)
write.csv(log_returns, file = "log_returns.csv", row.names = FALSE)